# Tableau Server Client 

TSC is library that provides a pythonic wrapper for interacting with your Tableau Server. It provides useful methods for pulling down data and interacting with your Tableau Server Programmatically.

A few use cases:
- Pull down data on your users, projects, and workbooks, and views
- Subscribe users to views and workbooks 
- Download and update content such as workbooks

The rest of this notebook will be dedicated to showing you how to authenicate to the tableau server and examples of how to interact with various server objects.

[Tableau Server Client (Python)](https://tableau.github.io/server-client-python/docs/api-ref#users)  link to the documentation on TSC libary

# Authenticating to the Tableau Server
### Pulling Data & Building Dataframes

In [7]:
import tableauserverclient as TSC
import pandas as pd

# Create an Authentication object, probably best to use a site or server admin account
tableau_auth = TSC.TableauAuth(<username>, <password>, site = <site name>)

# Create a Server object to that you will authenticate and call methods on
server = TSC.Server(<tableau server path>) #ex. https://tableau.company_name.com

# Authenticate to the server and pull down dataframes of users, workbooks, & subscriptions
with server.auth.sign_in(tableau_auth):
    all_users = list(TSC.Pager(server.users)) # use list & pager to pull down all users
    users = pd.DataFrame(data={'name':[user.name for user in all_users],
                               'user_id':[user.id for user in all_users],
                               'email':[user.email for user in all_users],
                               'site_role':[user.site_role for user in all_users]})
    
    all_workbooks = list(TSC.Pager(server.workbooks))
    workbooks = pd.DataFrame(data={'name':[workbook.name for workbook in all_workbooks],
                                   'workbook_id':[workbook.id for workbook in all_workbooks],
                                   'owner_id':[workbook.owner_id for workbook in all_workbooks]})
    
    all_subs = list(TSC.Pager(server.subscriptions))
    subscription = pd.DataFrame(data = {'sub_id':[sub.id for sub in all_subs],
                                        'subject': [sub.subject for sub in all_subs],
                                        'sub_schedule_id': [sub.schedule_id for sub in all_subs],
                                        'sub_user_id': [sub.user_id for sub in all_subs],
                                        'view/workbook_id': [sub.target.id for sub in all_subs],
                                        'target_type': [sub.target.type for sub in all_subs]})

## Example of building a subscription
This can be used with loops to build a large number of subscriptions or programatically add folks

In [ ]:
# First build a target for the subscription
target_view = TSC.Target(<view_id/workbook_id>,<target_type>)

with server.auth.sign_in(tableau_auth):
    new_sub = TSC.SubscriptionItem(subject=<subject>, 
                                   schedule_id=<schedule>, 
                                   user_id=<user_id>, 
                                   target=target_view)
    server.subscriptions.create(new_sub)